<a href="https://colab.research.google.com/github/geonextgis/End-to-End-Deep-Learning/blob/main/01_ANN/06_Backpropagation_on_Regression_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Backpropagation on Regression Data**
Backpropagation, short for "backward propagation of errors," is a supervised learning algorithm used to train artificial neural networks, which are a fundamental component of deep learning. The purpose of backpropagation is to minimize the error between the predicted output of the neural network and the actual target output by adjusting the network's weights and biases.

Here's a step-by-step explanation of how backpropagation works:

1. **Forward Pass:**
   - The input data is passed through the neural network to generate a predicted output.
   - Each neuron in the network performs a weighted sum of its inputs, applies an activation function to the result, and passes the output to the next layer.

2. **Calculate Error:**
   - The predicted output is compared to the actual target output to calculate the error. Common loss functions, such as mean squared error or cross-entropy, are used for this purpose.

3. **Backward Pass (Backpropagation):**
   - The error is then propagated backward through the network to update the weights and biases.
   - The partial derivative of the error with respect to each weight and bias is computed using the chain rule of calculus. This indicates how much the error would increase or decrease with a small change in each weight and bias.
   - The weights and biases are adjusted in the opposite direction of the gradient to minimize the error.

4. **Update Weights and Biases:**
   - The learning rate is applied to control the size of the weight and bias updates.
   - The weights and biases are updated based on the calculated gradients, nudging them towards values that reduce the error.

5. **Iterative Process:**
   - Steps 1-4 are repeated iteratively for multiple epochs until the neural network converges to a set of weights and biases that minimize the error on the training data.

The backpropagation algorithm allows neural networks to learn from data by adjusting their parameters to improve performance. It's a crucial component of training deep learning models and has been instrumental in the success of various applications, including image and speech recognition, natural language processing, and many others.

<center><img src="https://miro.medium.com/v2/resize:fit:1280/1*VF9xl3cZr2_qyoLfDJajZw.gif" width="50%"></center>


## **Import Required Libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense
print(tf.__version__)

import warnings
warnings.filterwarnings("ignore")

2.15.0


## **Create a DataFrame**

In [2]:
df = pd.DataFrame([[8, 8, 4], [7, 9, 5], [6, 10, 6], [5, 12, 17]], columns=["cgpa", "profile_score", "lpa"])
df

,cgpa,profile_score,lpa
0,8,8,4
1,7,9,5
2,6,10,6
3,5,12,17


## **Build the Algorithm**

In [3]:
# Create a function to initialize the weights and biases
def initialize_parameters(layer_dims):

    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)

    for l in range(1, L):

        parameters["W" + str(l)] = np.ones((layer_dims[l-1], layer_dims[l])) * 0.1
        parameters["b" + str(l)] = np.zeros((layer_dims[l], 1))

    return parameters

In [4]:
# Check the function
initialize_parameters([2, 2, 1])

{'W1': array([[0.1, 0.1],
        [0.1, 0.1]]),
 'b1': array([[0.],
        [0.]]),
 'W2': array([[0.1],
        [0.1]]),
 'b2': array([[0.]])}

In [5]:
# Write a function to linearly forward the data
def linear_forward(A_prev, W, b):

    Z = np.dot(W.T, A_prev) + b

    return Z

In [6]:
# Write a function for Forward Propagation
def L_layer_forward(X, parameters):

    A = X
    L = len(parameters) // 2 # Number of layers in the neural network

    for l in range(1, L+1):
        A_prev = A
        Wl = parameters["W" + str(l)]
        bl = parameters["b" + str(l)]
        # print("A"+str(l-1)+": ", A_prev)
        # print("W"+str(l)+": ", Wl)
        # print("b"+str(l)+": ", bl)
        # print("__"*20)

        A = linear_forward(A_prev, Wl, bl)
        # print("A" + str(l)+": ", A)
        # print("**"*20)

    return A, A_prev

In [7]:
# Extract the X and y value of a particular student
X = df[["cgpa", "profile_score"]].values[0].reshape(2, 1) # Shape(no of features, no. of training example)
y = df[["lpa"]].values[0][0]

# Initialize the parameters
parameters = initialize_parameters([2, 2, 1])

# Apply forward propagation on that particular student
y_hat, A1 = L_layer_forward(X, parameters)

In [8]:
y_hat = y_hat[0][0]
print(y_hat)
print(A1)
print("Loss:", (y-y_hat)**2)

0.32000000000000006
[[1.6]
 [1.6]]
Loss: 13.542399999999997


In [9]:
# Write a function to update the parameters
def update_parameters(parameters, y, y_hat, A1, X, lr=0.001):
    parameters["W2"][0][0] = parameters["W2"][0][0] + (lr * 2 * (y - y_hat) * A1[0][0])
    parameters["W2"][1][0] = parameters["W2"][1][0] + (lr * 2 * (y - y_hat) * A1[1][0])
    parameters["b2"][0][0] = parameters["b2"][0][0] + (lr * 2 * (y - y_hat))

    parameters["W1"][0][0] = parameters["W1"][0][0] + (lr * 2 * (y - y_hat) * parameters["W2"][0][0] * X[0][0])
    parameters["W1"][0][1] = parameters["W1"][0][1] + (lr * 2 * (y - y_hat) * parameters["W2"][0][0] * X[1][0])
    parameters["b1"][0][0] = parameters["b1"][0][0] + (lr * 2 * (y - y_hat) * parameters["W2"][0][0])

    parameters["W1"][1][0] = parameters["W1"][1][0] + (lr * 2 * (y - y_hat) * parameters["W2"][1][0] * X[0][0])
    parameters["W1"][1][1] = parameters["W1"][1][1] + (lr * 2 * (y - y_hat) * parameters["W2"][1][0] * X[1][0])
    parameters["b1"][1][0] = parameters["b1"][1][0] + (lr * 2 * (y - y_hat) * parameters["W2"][1][0])

In [10]:
# Apply the function to update the parameters value
update_parameters(parameters, y, y_hat, A1, X, lr=0.001)
parameters

{'W1': array([[0.10658137, 0.10658137],
        [0.10658137, 0.10658137]]),
 'b1': array([[0.00082267],
        [0.00082267]]),
 'W2': array([[0.111776],
        [0.111776]]),
 'b2': array([[0.00736]])}

In [11]:
# Epoch implementation
parameters = initialize_parameters([2, 2, 1])
epochs = 5

for i in range(epochs):

    Loss = []

    for j in range(df.shape[0]):
        X = df[["cgpa", "profile_score"]].values[j].reshape(2, 1)
        y = df[["lpa"]].values[j][0]

        # Parameter initialization
        y_hat, A1 = L_layer_forward(X, parameters)
        y_hat = y_hat[0][0]

        update_parameters(parameters, y, y_hat, A1, X)

        Loss.append((y - y_hat)**2)
    print(f"Epoch - {i+1}, Loss - {np.array(Loss).mean()}")

parameters

Epoch - 1, Loss - 82.89265194890827
Epoch - 2, Loss - 62.34912590789556
Epoch - 3, Loss - 35.54087621847292
Epoch - 4, Loss - 32.81500942310105
Epoch - 5, Loss - 37.37692144074401


{'W1': array([[0.30543903, 0.67317661],
        [0.34637222, 0.79853514]]),
 'b1': array([[0.04550973],
        [0.05514902]]),
 'W2': array([[0.50906895],
        [0.68356253]]),
 'b2': array([[0.14298969]])}

## **Backpropagation in Keras**

In [12]:
# Build the same model architecture using Keras
model = Sequential()

model.add(Dense(2, activation="linear", input_dim=2))
model.add(Dense(1, activation="linear"))

In [13]:
# Print the summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 6         
                                                                 
 dense_1 (Dense)             (None, 1)                 3         
                                                                 
Total params: 9 (36.00 Byte)
Trainable params: 9 (36.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Cehck the value of random weights initialized by Keras
model.get_weights()

[array([[-0.8022482 , -0.15731835],
        [ 1.1564075 ,  0.62193644]], dtype=float32),
 array([0., 0.], dtype=float32),
 array([[ 0.28612888],
        [-0.77632874]], dtype=float32),
 array([0.], dtype=float32)]

In [15]:
# Set the initial weights to 0.1 and biases to 0
new_weights = [np.array([[0.1 , 0.1 ],
                         [0.1 , 0.1]], dtype=np.float32),
               np.array([0., 0.], dtype=np.float32),
               np.array([[ 0.1],
                         [0.1 ]], dtype=np.float32),
               np.array([0.], dtype=np.float32)]

model.set_weights(new_weights)
model.get_weights()

[array([[0.1, 0.1],
        [0.1, 0.1]], dtype=float32),
 array([0., 0.], dtype=float32),
 array([[0.1],
        [0.1]], dtype=float32),
 array([0.], dtype=float32)]

In [16]:
# Define the optimizer and compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss="mean_squared_error", optimizer=optimizer)

In [19]:
# Fit the training data
history = model.fit(df.iloc[:, 0:-1].values, df["lpa"].values, epochs=1000, verbose=1, batch_size=1)

Epoch 1/1000
4/4 [==============================] - 0s 4ms/step - loss: 5.0087
Epoch 2/1000
4/4 [==============================] - 0s 4ms/step - loss: 4.9236
Epoch 3/1000
4/4 [==============================] - 0s 4ms/step - loss: 5.0002
Epoch 4/1000
4/4 [==============================] - 0s 4ms/step - loss: 4.9529
Epoch 5/1000
4/4 [==============================] - 0s 4ms/step - loss: 4.9517
Epoch 6/1000
4/4 [==============================] - 0s 5ms/step - loss: 4.9429
Epoch 7/1000
4/4 [==============================] - 0s 4ms/step - loss: 4.9107
Epoch 8/1000
4/4 [==============================] - 0s 3ms/step - loss: 4.9179
Epoch 9/1000
4/4 [==============================] - 0s 4ms/step - loss: 5.0047
Epoch 10/1000
4/4 [==============================] - 0s 4ms/step - loss: 4.9591
Epoch 11/1000
4/4 [==============================] - 0s 4ms/step - loss: 4.9457
Epoch 12/1000
4/4 [==============================] - 0s 4ms/step - loss: 4.9919
Epoch 13/1000
4/4 [==============================